In [1]:
import requests
import pickle
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

In [8]:
my_dict = dict()
file_path_txt = 'output.txt'
art = "3C9885405GMVQK"


chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = uc.Chrome(options=chrome_options)
driver.get("https://emex.ru")


with open(file_path_txt, 'r') as file:
    data = file.readlines()
    
try:
    
    click_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div/div/div[2]/div[1]/div[1]/div[2]/div/div/div')
    
    click_element.click()
    input_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div/div/div[2]/div[1]/div[1]/div[2]/div/div/input')
    input_element.send_keys(art)
    
    actions = ActionChains(driver) 
    actions.send_keys(Keys.ENTER)
    actions.perform()
    
    time.sleep(5)
    ymap = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div[4]/div[1]/ymaps/ymaps/ymaps/ymaps[1]')
    button_resize = ymap.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[4]/div[1]/ymaps/ymaps/ymaps/ymaps[4]/ymaps[3]/ymaps/ymaps/ymaps[1]/ymaps')
    button_resize.click()
    button_resize.click()
    
    time.sleep(7)
    element_deal = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div[2]/div/div/div[1]/div[2]/div/div/div/div[1]/div[1]/a')
    element_deal.click()
    
    translator = str.maketrans("", "", "\n")
    time.sleep(3)
    for item in data:
        
        item = item.translate(translator)
        
        input_element = driver.find_element(By.CLASS_NAME, 'e-inputField')
        input_element.clear()
        input_element.send_keys(item)
        
        time.sleep(3)
        actions = ActionChains(driver) 
        actions.send_keys(Keys.ENTER)
        actions.perform()
        
        time.sleep(3)
        try:
            
            same_art_elements = driver.find_elements(By.CLASS_NAME, 'sc-6a057c55-1')
            name_element = driver.find_element(By.CLASS_NAME, 'sc-e789b6e9-0').text
            
            for same_art_element in same_art_elements:
            
                detale_elements = same_art_element.find_elements(By.CLASS_NAME, 'sc-6e4a9aa6-3')
                
                for element in detale_elements:
                    
#                     time.sleep(1)
                    link_elements = element.find_elements(By.TAG_NAME, 'a')
                
                    list_items = []
                    
                    for text in link_elements:
                        
                        list_items.append(text.text)
                    if item in my_dict:
                        my_dict[item].append(list_items)
                    else:  
                        my_dict[item] = [list_items]
                
        except (NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException) as e:
            
            print(item + ": no same arts in system")
            continue    

except Exception as e:
    
    print(f"An error occurred during the request: {str(e)}")
        
for key, value in my_dict.items():
    print(f"{key}: {value}")        

In [96]:
file_path_txt = 'all_arts_8396.txt'

with open(file_path_txt, 'r') as file:
    data = file.readlines()
    
translator = str.maketrans("", "", "\n")    

for item in data:
    item = item.translate(translator)
    print(item)

6Q0820129
6C0419685A
B000750M7
3C2867172B21X
3C9885405GMVQK
6C0853688B
809-075
N10247504
3C2868157A95T


In [86]:
base_url = "https://emex.ru/products/"
file_path_txt = 'all_arts_8396.txt'

translator = str.maketrans("", "", "\n")

driver = uc.Chrome(chrome_options=chrome_options)

with open(file_path_txt, 'r') as file:
    data = file.readlines()
    
for item in data:   
    
    item = item.translate(translator)
    url = base_url + str(item)
    print(url)
    driver.get(url)
    time.sleep(3)
#     try:

#         same_art_elements = driver.find_elements(By.CLASS_NAME, 'sc-f2a13e83-0 jVNWio')
#         print("same_art_elements")

#         for same_art_element in same_art_elements:

#             detale_elements = same_art_element.find_elements(By.CLASS_NAME, 'sc-6e4a9aa6-3 sBrbz')
#             print("detale_elements")

#             for element in detale_elements:

#                 time.sleep(3)
#                 link_elements = element.find_elements(By.TAG_NAME, 'a')
#                 print("link_elements")
#                 list_items = []

#                 for text in link_elements:
#                     print("text")
#                     list_items.append(text)

#                 my_dict[item] = list_items

#     except (NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException) as e:

#         print(item + ": no same arts in system")
#         continue    
        
#     driver.close()

https://emex.ru/products/6Q0820129
https://emex.ru/products/6C0419685A
https://emex.ru/products/B000750M7
https://emex.ru/products/3C2867172B21X
https://emex.ru/products/3C9885405GMVQK
https://emex.ru/products/6C0853688B
https://emex.ru/products/809-075
https://emex.ru/products/N10247504


KeyboardInterrupt: 